<a href="https://colab.research.google.com/github/StevenFromUnimiMIEDSE/Financial_Market_Data/blob/main/EuriborFromECB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtaining Euribor data from public API
Source of methods and ideas:
1. https://stackoverflow.com/questions/75890191/obtain-euribor-rates-with-ecb-sdmx-rest-api
2. https://data.ecb.europa.eu/data/datasets/FM?dataset%5B0%5D=Financial%20market%20data%20%28FM%29&filterSequence=dataset&advFilterDataset%5B0%5D=Financial%20market%20data%20%28FM%29


Extract recent euribor trend 3M

In [1]:
import requests
import pandas as pd
import plotly.express as px

# Set the URL for the data
url = "https://data-api.ecb.europa.eu/service/data/FM/M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA?lastNObservations=24&detail=dataonly&format=jsondata"

# Fetch the data
response = requests.get(url)
data = response.json()

# Extract observations
observations = data['dataSets'][0]['series']['0:0:0:0:0:0:0']['observations']
dates = [data['structure']['dimensions']['observation'][0]['values'][i]['id'] for i in range(len(observations))]
values = [observations[str(i)][0] for i in range(len(observations))]

# Create a DataFrame
euribor_data = pd.DataFrame({'Date': pd.to_datetime(dates, format='%Y-%m'), 'EuriborRate': values})

# Plotting
fig = px.line(euribor_data, x='Date', y='EuriborRate', title='Recent Euribor Trend', markers=True)
fig.show()

1M 3M 6M

In [2]:
import requests
import pandas as pd
import plotly.graph_objects as go

# Base URL parts
base_url = "https://data-api.ecb.europa.eu/service/data/FM/M.U2.EUR.RT.MM."
euribor_types = ["EURIBOR1MD_", "EURIBOR3MD_", "EURIBOR6MD_"]
last_n_observations = "24"

# Initialize a DataFrame
euribor_data = pd.DataFrame()

# Fetch data for each Euribor type and add to the DataFrame
for euribor_type in euribor_types:
    full_url = f"{base_url}{euribor_type}.HSTA?lastNObservations={last_n_observations}&detail=dataonly&format=jsondata"
    response = requests.get(full_url)
    data = response.json()
    observations = data['dataSets'][0]['series']['0:0:0:0:0:0:0']['observations']
    dates = [data['structure']['dimensions']['observation'][0]['values'][i]['id'] for i in range(len(observations))]
    values = [observations[str(i)][0] for i in range(len(observations))]
    temp_df = pd.DataFrame({'Date': pd.to_datetime(dates, format='%Y-%m'), f'EuriborRate_{euribor_type}': values})
    euribor_data = pd.merge(euribor_data, temp_df, on='Date', how='outer') if not euribor_data.empty else temp_df

# Sorting by Date
euribor_data = euribor_data.sort_values('Date')

# Plotting
fig = go.Figure()
for euribor_type in euribor_types:
    fig.add_trace(go.Scatter(x=euribor_data['Date'], y=euribor_data[f'EuriborRate_{euribor_type}'], mode='lines+markers', name=euribor_type))
fig.update_layout(title='Recent Euribor Trends', xaxis_title='Date', yaxis_title='Euribor Rate', legend_title='Euribor Type')
fig.show()
